# Deep Learning

# Utilizing Apache Spark and PySpark for Deep Learning in Neuroimaging Studies

## Why Spark?

- **Efficient Large Data Handling:**
   - Spark is designed for fast computation and handling big datasets, essential for neuroimaging data.

- **PySpark - Python Integration:**
   - PySpark provides a Python interface to Spark, making it easier to implement deep learning algorithms using Python libraries.

- **Distributed Computing:**
   - Spark's distributed nature allows for parallel processing, speeding up the deep learning tasks.

- **MLlib for Machine Learning:**
   - Spark's MLlib offers machine learning algorithms optimized for big data, which we can leverage for our deep learning models.

## Advantages:

- Spark handles our complex and large participant data efficiently.
- PySpark allows seamless integration with Python's deep learning libraries.
- Distributed processing capability accelerates our deep learning computations.
- Scalability ensures that our system can grow with our data needs.

In summary, Apache Spark and PySpark provide a robust, scalable, and efficient platform for deep learning in our neuroimaging study, enabling us to process large datasets with speed and ease.


## 1 - Setting up Spark cluster

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("PySpark_VSCode").getOrCreate()


In [7]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import models
from keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input


In [8]:
combined_df = pd.read_csv('/home/skander/combined.csv')

In [9]:

# Set 'participant_id' as index
combined_df_with_index = combined_df.set_index('participant_id')

# Extract features for pattern discovery
features = combined_df_with_index  # All columns except 'participant_id'

# Now features DataFrame is ready for unsupervised learning techniques



## 2 - Neuron layer

In [11]:
# Adjust the input dimension to match the number of features in your dataset
input_dim = features.shape[1]  # Number of features

# Define the size of the encoded representations
encoding_dim = 32  # experiment with this value

# Input layer
input_layer = Input(shape=(input_dim,))

# Encoded layer - where the data is compressed
encoded = Dense(encoding_dim, activation='relu')(input_layer)

# Decoded layer - reconstruction of the input
decoded = Dense(input_dim, activation='sigmoid')(encoded)

# Autoencoder model
autoencoder = Model(input_layer, decoded)

# Compile the model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Train the autoencoder
autoencoder.fit(features, features,  # Input and output are the same for an autoencoder
                epochs=50,          # Number of epochs
                batch_size=256,     # Batch size
                shuffle=True)       # Shuffle the data

Epoch 1/50


2024-02-01 09:00:40.486215: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-01 09:00:40.585502: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


1/1 [==============================] - 0s 207ms/step - loss: 3.8192
Epoch 2/50
1/1 [==============================] - 0s 2ms/step - loss: 2.1674
Epoch 3/50
1/1 [==============================] - 0s 1ms/step - loss: 0.5199
Epoch 4/50
1/1 [==============================] - 0s 1ms/step - loss: -1.1258
Epoch 5/50
1/1 [==============================] - 0s 1ms/step - loss: -2.7679
Epoch 6/50
1/1 [==============================] - 0s 1ms/step - loss: -4.4042
Epoch 7/50
1/1 [==============================] - 0s 2ms/step - loss: -6.0364
Epoch 8/50
1/1 [==============================] - 0s 1ms/step - loss: -7.6663
Epoch 9/50
1/1 [==============================] - 0s 2ms/step - loss: -9.2947
Epoch 10/50
1/1 [==============================] - 0s 1ms/step - loss: -10.9216
Epoch 11/50
1/1 [==============================] - 0s 1ms/step - loss: -12.5484
Epoch 12/50
1/1 [==============================] - 0s 1ms/step - loss: -14.1733
Epoch 13/50
1/1 [==============================] - 0s 1ms/step - loss: